# Hollow

Jupyter Notebook for simple working with [*Hollow*](https://boscoh.github.io/hollow/) software.

In [ ]:
#@title Prepare the environment
#@markdown Execute this cell before running the following cells. Only need to be done once, the rest can be run multiple times.
%%capture

!git clone https://github.com/boscoh/hollow

import subprocess
from urllib import request

try:
    from google.colab import files, widgets
    google_colab = True
except ImportError:
    google_colab = False

In [ ]:
#@title Upload pdb
#@markdown Write one or several PDB codes (comma separated) to perform the calculation.
#@markdown Alternatively, upload one or several pdb files from your local computer.

PDB = ''  # @param {type:"string"}

pdb_files = []
if PDB.strip():
    pdbs = PDB.split(',')
    for pdb in pdbs:
        if pdb := pdb.strip():
            request.urlretrieve(f'https://files.rcsb.org/download/{pdb}.pdb', f'{pdb}.pdb')
            pdb_files.append(f'{pdb}.pdb')
elif google_colab:
    uploaded = files.upload()
    for filename, filestr in uploaded.items():
        pdb_files.append(filename)

In [ ]:
#@title Hollow parameters and calculation
#@markdown Set the parameters for the hollow calculation.  
#@markdown The hollow calculation will be performed for each pdb file uploaded individually. The results will be downloaded at the end.

#@markdown Grid spacing in angstroms (default 0.5Å; suggested 0.2Å for final resolution)
GRID_SPACING = 0.5  # @param {type:"number"}

#@markdown Radius of ball to explore cavities (default 1.4Å = 95% x radius of output atom type suggested)
INTERIOR_PROBE = 1.4  # @param {type:"number"}

#@markdown Radius of probe to roll over surface used to define depressions (default 8.0Å)
SURFACE_PROBE = 8.0  # @param {type:"number"}

#@markdown Process water molecules
WATER = False  # @param {type:"boolean"}

#@markdown Radius around a grid point, in which the b-factors of heavy atoms are averaged (0.0=off; suggested=4.0; default=0)
BFACTOR_PROBE = 0.0  # @param {type:"number"}

#@markdown Constrained mode: define a sphere (around atom1) or cylinder (from atom1 to atom2, with offsets) to limit the search
CONSTRAINT = ""  # @param ["", "sphere", "cylinder"]
RADIUS = 13.0  # @param {type:"number"}
CHAIN1 = 'A'  # @param {type:"string"}
RES1 = 1  # @param {type:"integer"}
ATOM1 = 'CA'  # @param {type:"string"}
OFFSET1 = 0.0  # @param {type:"number"}
CHAIN2 = 'A'  # @param {type:"string"}
RES2 = 2  # @param {type:"integer"}
ATOM2 = 'CA'  # @param {type:"string"}
OFFSET2 = 0.0  # @param {type:"number"}


if CONSTRAINT:
    with open('constraints.txt', 'w') as f:
        f.write("{\n")
        f.write(f"  'type': '{CONSTRAINT}',\n")
        f.write(f"  'remove_asa_shell': True,\n")
        f.write(f"  'radius': {RADIUS:.2f},\n")
        f.write(f"  'chain1': '{CHAIN1:s}',\n")
        f.write(f"  'res_num1': {RES1:d},\n")
        f.write(f"  'atom1': '{ATOM1:s}',\n")
        if CONSTRAINT == 'cylinder':
            f.write(f"  'axis_offset1': {OFFSET1:d},\n")
            f.write(f"  'chain2': '{CHAIN2:s}',\n")
            f.write(f"  'res_num2': {RES2:d},\n")
            f.write(f"  'atom2': '{ATOM2:s}',\n")
            f.write(f"  'axis_offset2': {OFFSET2:d},\n")
        f.write("}\n")

#TODO: fix import mess and use make_hollow_spheres function
pdb_hollow_files = []
for pdb in pdb_files:
    print(f"####  Running HOLLOW for '{pdb}'")
    hollow_args = f'-g {GRID_SPACING:.2f} -p {INTERIOR_PROBE:.2f} -s {SURFACE_PROBE:.2f} {f"-w" if WATER else ""} -b {BFACTOR_PROBE:.2f} {f"-c constraints.txt" if CONSTRAINT else ""} {pdb}'
    hollow_run = subprocess.run(f'python3 hollow/hollow.py {hollow_args}', shell=True, capture_output=True)
    print(hollow_run.stdout.decode('utf-8'))
    if hollow_run.returncode == 0:
        pdb_hollow_files.append(pdb.replace('.pdb', '-hollow.pdb'))
    else:
        print(f"####  ERROR: HOLLOW failed for '{pdb}'")

for pdb_hollow in pdb_hollow_files:
    if google_colab:
        files.download(pdb_hollow)